# Vocab Study Notebook

In [1]:
from iMahir import Study

In [2]:
hebrew = '/Users/cody/Documents/vocabulary/hebrew_NEW.json'

In [3]:
heb = Study(hebrew)

preparing TF...
TF app is up-to-date.
Using annotation/app-Bhsa commit 7f353d587f4befb6efe1742831e28f301d2b3cea (=latest)
  in /Users/cody/text-fabric-data/__apps__/Bhsa.


hebrew ready for study.
this is session 6:
score 1: 9 terms
score 2: 29 terms
score 3: 187 terms
total: 225


In [4]:
heb.learn()

 q


Exception: Quit initiated. Nothing saved.

### Notes

* add "sheath" gloss to תער
* add "to help" to עות
* add "to feel" to חושׁ
* Check עטף, which has 3 roots. Two other glosses include: "to turn aside; to envelop oneself"
* חרץ add "to decide", also look at "to be yellow"
* PLG.n add "stream"
* check BDB for קיץ.v for gloss of "to awake"
* add "stink" to ZNX.v